# 기본 설정

# colab 용

In [1]:
# Google Drive Mount

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd gdrive/My Drive/스스로공부하는능동적이고자율적인혀누

/content/gdrive/My Drive/스스로공부하는능동적이고자율적인혀누


# 로컬용

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# 지도시각화
import folium
from folium import plugins
import html
import json
from pandas.io.json import json_normalize
import os
import requests
import ipywidgets
from IPython.display import Image
from ipywidgets import interact
pd.set_option("display.max_rows", 220)
pd.set_option("display.max_columns", 220)

# ver 3.0로 하기

In [2]:
node = pd.read_csv('node v3.0.csv')
phar = pd.read_csv('약국정보_운영시간 ver2.0.csv')

In [3]:
## 거리 구하는 함수
import numbers
import math

class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

    @staticmethod
    def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
        """
        유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        dLon = abs(x2-x1) # 경도 차이
        if dLon >= 180:   # 반대편으로 갈 수 있는 경우
            dLon -= 360   # 반대편 각을 구한다
        dLat = y2-y1      # 위도 차이
        return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)

In [4]:
def jsonify(data):
    df = json_normalize(data, '근처약국', ['날짜','시간'])
    df = df[['날짜', '시간','node_id','행정동', 'near_phar']]
    return df

In [ ]:
phar[['날짜', '요일']].drop_duplicates().sort_values(by=['날짜'])['날짜'].values[:27]   # 현우
phar[['날짜', '요일']].drop_duplicates().sort_values(by=['날짜'])['날짜'].values[27:54] # 지환
phar[['날짜', '요일']].drop_duplicates().sort_values(by=['날짜'])['날짜'].values[54:81] # 건욱
phar[['날짜', '요일']].drop_duplicates().sort_values(by=['날짜'])['날짜'].values[81:]   # 세은

In [39]:
total_data = []
### 밑에 코드 주석처리에 위 cell에 있는 자신의 할당량을 넣으시면 되뮤 ###
for target_date in tqdm(phar[['날짜', '요일']].drop_duplicates().sort_values(by=['날짜'])['날짜'].values[9:27]):
    for target_time in tqdm(range(6, 25)):
        df_temp_phar = phar[phar['날짜']== target_date]
        df_temp_phar = df_temp_phar[df_temp_phar[str(target_time)] == 1]  # 해당 날짜와 시간에 근무하는 약국 찾기

        list_node = list(node['node_id'].unique())
        list_phar_add = list(df_temp_phar['주소'].unique())
        list_near_phar = []
        for eachNode in list_node:
            cri_node_lon = node[node['node_id']==eachNode]['node_lon'].values[0]
            cri_node_lat = node[node['node_id']==eachNode]['node_lat'].values[0]
            list_target = list_node.copy()
            list_target.remove(eachNode)

            # node와 약국 사이의 거리 구하기
            list_collect_distance = []
            dict_num_of_phar = {}
            for phar_add in list_phar_add:
                target_lon = df_temp_phar[df_temp_phar['주소']==phar_add]['Longitude'].values[0]
                target_lat = df_temp_phar[df_temp_phar['주소']==phar_add]['Latitude'].values[0]
                list_collect_distance.append(GeoUtil.get_harversion_distance(cri_node_lon, cri_node_lat, target_lon, target_lat))
            num_of_phar = len([dist for dist in list_collect_distance if dist <= 0.5])
            dict_num_of_phar['node_id'] = eachNode
            dict_num_of_phar['행정동'] = node[node['node_id']==eachNode]['행정동'].values[0]
            dict_num_of_phar['near_phar'] = num_of_phar
            list_near_phar.append(dict_num_of_phar)

        dict_result = {}
        dict_result['날짜'] = target_date
        dict_result['시간'] = target_time
        dict_result['근처약국'] = list_near_phar      

        total_data.append(dict_result)

 21%|██        | 4/19 [01:03<04:25, 17.70s/it]


100%|██████████| 18/18 [12:51:50<00:00, 2572.82s/it]


In [40]:
df = jsonify(total_data)
df

<ipython-input-4-19df07b17dbd>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(data, '근처약국', ['날짜','시간'])


,날짜,시간,node_id,행정동,near_phar
0,2019년 11월 19일,6,0,산내동,0
1,2019년 11월 19일,6,1,산내동,0
2,2019년 11월 19일,6,2,산내동,0
3,2019년 11월 19일,6,3,산내동,0
4,2019년 11월 19일,6,4,산성동,0
...,...,...,...,...,...
188095,2019년 1월 22일,24,545,덕암동,0
188096,2019년 1월 22일,24,546,석봉동,0
188097,2019년 1월 22일,24,547,석봉동,0
188098,2019년 1월 22일,24,548,석봉동,0


In [38]:
df1

,날짜,시간,node_id,행정동,near_phar
0,2019년 10월 10일,6,0,산내동,0
1,2019년 10월 10일,6,1,산내동,0
2,2019년 10월 10일,6,2,산내동,0
3,2019년 10월 10일,6,3,산내동,0
4,2019년 10월 10일,6,4,산성동,0
...,...,...,...,...,...
104495,2019년 11월 19일,24,545,덕암동,0
104496,2019년 11월 19일,24,546,석봉동,0
104497,2019년 11월 19일,24,547,석봉동,0
104498,2019년 11월 19일,24,548,석봉동,0


In [32]:
phar[['날짜', '요일']].drop_duplicates().sort_values(by=['날짜'])['날짜'].values[9:27]

array(['2019년 11월 19일', '2019년 11월 20일', '2019년 11월 21일', '2019년 11월 22일',
       '2019년 11월 23일', '2019년 12월 10일', '2019년 12월 11일', '2019년 12월 12일',
       '2019년 12월 13일', '2019년 12월 14일', '2019년 12월 22일', '2019년 12월 23일',
       '2019년 12월 24일', '2019년 12월 8일', '2019년 12월 9일', '2019년 1월 20일',
       '2019년 1월 21일', '2019년 1월 22일'], dtype=object)

# 0~27까지 merge해주기

In [43]:
df1 = pd.read_csv('0~9 2019년 11월 19일까지.csv')
df2 = pd.read_csv('9~27 2019년 1월 22일까지.csv')

In [46]:
df = pd.concat([df1,df2])

In [48]:
df['날짜'].nunique()

27

In [49]:
df

,날짜,시간,node_id,행정동,near_phar
0,2019년 10월 10일,6,0,산내동,0
1,2019년 10월 10일,6,1,산내동,0
2,2019년 10월 10일,6,2,산내동,0
3,2019년 10월 10일,6,3,산내동,0
4,2019년 10월 10일,6,4,산성동,0
...,...,...,...,...,...
188095,2019년 1월 22일,24,545,덕암동,0
188096,2019년 1월 22일,24,546,석봉동,0
188097,2019년 1월 22일,24,547,석봉동,0
188098,2019년 1월 22일,24,548,석봉동,0


In [47]:
df.to_csv('0~27 2019년 1월 22일까지.csv', index=False)